In [1]:
import pandas as pd
import numpy as np
from snorkel.labeling import labeling_function
import re
import editdistance as ed
from sklearn.model_selection import train_test_split
from spacy.lang.en import English
import nltk
nltk.download('wordnet')
from nltk import word_tokenize
nltk.download('punkt')
from nltk.corpus import wordnet 
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import spacy
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.stem import LancasterStemmer
from snorkel.labeling import PandasLFApplier, LFApplier, LFAnalysis
from snorkel.analysis import get_label_buckets

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\I517193\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\I517193\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Schema
* The data consist of the queries users searched for, by voice, converted to text
* All queries are labeled with their tags (each unigram is labeled with the entity name)
* The entities covered are as follows:
  * Restuarant_name
  * Rating
  * Amenity
  * Location
  * Price
  * Hours
  * Dish
  * Cuisine

**Preprocessing the file**

* Separating each candidate noun phrase from the query

In [2]:
def ret_df_file(file):
    with open(file, "r") as f:
        lines = f.readlines()
        size = len(lines) 
        idx_list = [idx + 1 for idx, val in
                enumerate(lines) if val == '\n'] 
        res = [lines[i: j-1] for i, j in
            zip([0] + idx_list, idx_list + 
            ([size] if idx_list[-1] != size else []))] 
        queries = ['' for i in range(len(res))]
        tags = []
        unigrams = []
        words = []
        poss = []
        for i, ele in enumerate(res):
            flag = False
            tempWords = []
            temT = []
            temW = []
            temPos = []
            for j, actual_str in enumerate(ele):
                res[i][j] = actual_str.split('\t')
                res[i][j][-1] = res[i][j][-1].replace('\n', '')
                if res[i][j][0][:2]=="B-" or res[i][j][0][:2]=="I-":
                    if flag:
                        if tempTag == res[i][j][0][2:]:
                            tempWords.append(res[i][j][-1])
                        else:
                            temT.append(tempTag)
                            temW.append(' '.join(tempWords))
                            pos[-1] = j-1
                            temPos.append(pos)
                            tempWords = []
                            pos = [j,j]
                            tempWords.append(res[i][j][-1])
                            tempTag = res[i][j][0][2:]
                        # tempTag.append
                    else:
                        tempWords = []
                        tempWords.append(res[i][j][-1])
                        pos = [j,j]
                        tempTag = res[i][j][0][2:]
                        flag = True
    #             elif res[i][j][0][:2]=="I-":
    #                 tempWords.append(res[i][j][-1])
    #                 temT.append(res[i][j][0][2:])
    #                 temW.append(' '.join(tempWords))
    #                 flag = False
                else:
                    if flag:
                        temT.append(tempTag)
                        temW.append(' '.join(tempWords))
                        pos[-1] = j-1
                        temPos.append(pos)
                        flag = False
                if j!=0:
                    queries[i] = queries[i] + ' ' + res[i][j][-1]
                else:
                    queries[i] = res[i][j][-1]
            if flag:
                temT.append(tempTag)
                temW.append(' '.join(tempWords))
                pos[-1] = j
                temPos.append(pos)
                flag = False
            tags.append(temT)
            words.append(temW)
            poss.append(temPos)
    df_queries = []
    df_pos_start = []
    df_pos_end = []
    n_grams = []
    df_tags = []
    for i, ele in enumerate(res):
        for j in range(len(tags[i])):
            df_queries.append(queries[i])
            df_tags.append(tags[i][j])
            df_pos_start.append(poss[i][j][0])
            df_pos_end.append(poss[i][j][1])
            n_grams.append(words[i][j])
    return pd.DataFrame({"n_grams": n_grams, "Tag": df_tags, "Query": df_queries, "Pos_start": df_pos_start, "Pos_end":df_pos_end})

**Getting the train and test data**

In [3]:
df_train = ret_df_file("../Datasets/MITRestuarant/restauranttrain.bio")
df_test = ret_df_file("../Datasets/MITRestuarant/restauranttest.bio")

In [4]:
print(df_train.shape)
print(df_test.shape)

(15363, 5)
(3151, 5)


In [5]:
df_train.head(5)

,n_grams,Tag,Query,Pos_start,Pos_end
0,2 start,Rating,2 start restaurants with inside dining,0,1
1,inside dining,Amenity,2 start restaurants with inside dining,4,5
2,5 star,Rating,5 star resturants in my town,0,1
3,in my town,Location,5 star resturants in my town,3,5
4,hong kong,Restaurant_Name,98 hong kong restaurant reasonable prices,1,2


In [6]:
df_test.head(5)

,n_grams,Tag,Query,Pos_start,Pos_end
0,four star,Rating,a four star restaurant with a bar,1,2
1,with a,Location,a four star restaurant with a bar,4,5
2,bar,Amenity,a four star restaurant with a bar,6,6
3,asian,Cuisine,any asian cuisine around,1,1
4,around,Location,any asian cuisine around,3,3


In [7]:
def obtain_noun_chunks_limited(df_train):
    list_of_queries = df_train.Query.unique()
    queries_train = []
    tags_train = []
    phrases_train = []
    pos_S = []
    pos_E = []
    nlp = spacy.load("en_core_web_sm")
    for query in list_of_queries:
        doc = nlp(query)
        positions_start = list(df_train.Pos_start[df_train.Query==query])
        positions_end = list(df_train.Pos_end[df_train.Query==query])
        for noun in doc.noun_chunks:
            flag = False
            for i in range(len(positions_start)):
                # checking intersection>0 and extra words in noun_chunk not more than 2 and difference 
                # between the phrases of not more than 2 words
                if len(set(list(range(noun.start, noun.end))) & \
                       set(list(range(positions_start[i], positions_end[i]+1))))>0 \
                and (len(set(list(range(noun.start, noun.end))).difference(set(list(range(positions_start[i], \
                                                                                            positions_end[i]+1)))))<=2 \
                    and len(set(list(range(positions_start[i], positions_end[i]+1))).difference(set(list(range(noun.start, noun.end)))))<=2):
                    queries_train.append(query)
                    tags_train.append(list(df_train.Tag[np.logical_and(df_train.Query==query, \
                                                                  df_train.Pos_start==positions_start[i])])[0])
                    phrases_train.append(noun.text)
                    pos_S.append(noun.start)
                    pos_E.append(noun.end-1)
                    flag = True
                    break
    return pd.DataFrame({"n_grams": phrases_train, "Tag": tags_train, \
                                "Query": queries_train, "Pos_start": pos_S, "Pos_end":pos_E})

In [8]:
df_train_limited_chunks = obtain_noun_chunks_limited(df_train)

In [9]:
df_valid_limited_chunks = obtain_noun_chunks_limited(df_test)

In [10]:
df_valid_limited_chunks.head()

,n_grams,Tag,Query,Pos_start,Pos_end
0,a four star restaurant,Rating,a four star restaurant with a bar,0,3
1,a bar,Location,a four star restaurant with a bar,5,6
2,any asian cuisine,Cuisine,any asian cuisine around,0,2
3,any bbq places,Cuisine,any bbq places open before 5 nearby,0,2
4,any dancing establishments,Location,any dancing establishments with reasonable pri...,0,2


In [11]:
df_train_limited_chunks.Tag.unique()

array(['Rating', 'Amenity', 'Location', 'Hours', 'Dish', 'Cuisine',
       'Price', 'Restaurant_Name'], dtype=object)

**Defining the Labels**

In [12]:
RATING = 0
AMENITY = 1
LOCATION = 2
RESTUARANT_NAME = 3
HOURS = 4
DISH = 5
CUISINE = 6
PRICE = 7
NO_TAG = 8
ABSTAIN  = -1

In [13]:
import warnings
warnings.filterwarnings("ignore")
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Rating'] = RATING
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Amenity'] = AMENITY
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Location'] = LOCATION
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Restaurant_Name'] = RESTUARANT_NAME
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Hours'] = HOURS
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Dish'] = DISH
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Cuisine'] = CUISINE
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Price'] = PRICE

## Writing Labelling functions

**Labelling functions for Rating**

In [14]:
@labeling_function()
def lf_rating_star(x):
    # Returns a label of rating if pattern of digit star's found in the phrase
    query = x.Query.lower()
    if re.findall("[\d]+[\W]?star", query):
        return RATING
    else:
        return ABSTAIN

In [15]:
@labeling_function()
def lf_rating_adj(x):
    # Returns a label of rating if any adjective within the edit distance or the base form of adjective is found in the phrase
    adjectives = ['fresh', 'recommended', 'fair', 'recommendation', 'rated'
       'terrific', 'well', 'awesome', 'awards', 'authentic', 'pleasing', 'favorite',
        'lovely', 'really', 'decent', 'busy', 'superior','average'
       'famous', 'poor', 'decent', 'simple', 'real', 'popular', 'wonderful',
       'casual', 'perfect', 'massive', 'nearby', 'nice',
       'delicious', 'winning', 'favourite',  'negative', 'award', 'positive', 'fancy',
       'outstanding', 'good', 'amazing', 'recommend',
       'strong', 'value','incredible',
       'fantastic', 'classy', 'top', 'former', 'rated','reviews','horrible','terrible', 
       'local', 'excellent', 'place',
       'tasty', 'rate', 'high', 'great']
    lemmatiser = WordNetLemmatizer()
    for word in x.Query.lower().split():
        for adjective in adjectives:
            if ed.eval(adjective, word)<=1 or lemmatiser.lemmatize(word, pos = "a")==adjective:
                return RATING
    return ABSTAIN

In [16]:
@labeling_function()
def lf_rating_adj_syn(x):
    # Returns a label of rating if any word in the phrase is a synonym or an antonym of the mentioned words
    adjectives = ['fresh', 'recommended', 'fair', 'recommendation', 'rated'
       'terrific', 'well', 'awesome', 'awards', 'authentic', 'pleasing', 'favorite',
        'lovely', 'really', 'decent', 'busy', 'superior',
       'famous', 'poor', 'decent', 'simple', 'real', 'popular', 'wonderful',
       'casual', 'perfect', 'massive', 'nearby', 'nice',
       'delicious', 'winning', 'favourite',  'negative', 'award', 'positive', 'fancy',
       'outstanding', 'good', 'amazing', 'recommend',
       'strong', 'value','incredible',
       'fantastic', 'classy', 'top', 'former', 'rated','reviews','horrible','terrible', 
       'local', 'excellent', 'place',
       'tasty', 'rate', 'high', 'great']
    synonyms = [] 
    antonyms = [] 
    for adj in adjectives:
        for syn in wordnet.synsets(adj): 
            for l in syn.lemmas(): 
                synonyms.append(l.name()) 
                if l.antonyms(): 
                    antonyms.append(l.antonyms()[0].name()) 
    
    lemmatiser = WordNetLemmatizer()
    for word in x.Query.lower().split():
        for adjective in synonyms:
            if ed.eval(adjective, word)<=1 or lemmatiser.lemmatize(word, pos = "a")==adjective:
                return RATING
        for adjective in antonyms:
            if ed.eval(adjective, word)<=1 or lemmatiser.lemmatize(word, pos = "a")==adjective:
                return RATING
    return ABSTAIN

**Labelling functions for Amenity**

In [17]:
@labeling_function()
def lf_amenity_lem(x):
    # Returns a label of rating if pattern of digit star's found in the phrase
    amenities = ['formal', 'outdoor', 'friendly', 'parking', 'special'
       'quiet', 'smoking', 'atmosphere', 'anniversary', 'birthday', 'tourist', 'reservation']
    lemmatiser = WordNetLemmatizer()
    for word in x.Query.lower().split():
        for adjective in amenities:
            if ed.eval(adjective, word)<=1 or lemmatiser.lemmatize(word, pos = "a")==adjective:
                return AMENITY
    return ABSTAIN

In [18]:
@labeling_function()
def lf_amenity_syn(x):
    # Returns a label of rating if pattern of digit star's found in the phrase
    amenities = ['formal', 'outdoor', 'friendly', 'parking', 'special'
       'quiet', 'smoking', 'atmosphere', 'anniversary', 'birthday', 'tourist', 'reservation']

    synonyms = [] 
    antonyms = [] 
    for adj in amenities:
        for syn in wordnet.synsets(adj): 
            for l in syn.lemmas(): 
                synonyms.append(l.name()) 
                if l.antonyms(): 
                    antonyms.append(l.antonyms()[0].name()) 
    
    lemmatiser = WordNetLemmatizer()
    for word in x.Query.lower().split():
        for adjective in synonyms:
            if ed.eval(adjective, word)<=1 or lemmatiser.lemmatize(word, pos = "a")==adjective:
                return AMENITY
        for adjective in antonyms:
            if ed.eval(adjective, word)<=1 or lemmatiser.lemmatize(word, pos = "a")==adjective:
                return AMENITY
    return ABSTAIN

**Labeling function for Location**

In [19]:
@labeling_function()
def lf_location_prep(x):
    # Returns a label of location if the word before the phrase is a preposition mentioned
    if (x.Pos_start>=1 and x.Query.split()[x.Pos_start - 1].lower() in ['in', 'near', 'above', 'over', 'by', 'along', 'around']) or \
    (x.Pos_start>1 and x.Query.split()[x.Pos_start - 2].lower() in  ['in', 'near', 'above', 'over', 'by', 'along', 'on']):
        return LOCATION
    elif x.Pos_end + 1 <= (len(x.Query.strip().split()) - 1) \
    and x.Query.split()[x.Pos_end + 1].lower() in ['where']:
        return LOCATION
    else:
        return ABSTAIN

In [20]:
df_rating_amenity_train = df_train_limited_chunks[df_train_limited_chunks['Tag'].isin([0,1,2])]

In [21]:
df_train_lf_rating_amenity, df_test_lf_rating_amenity, y_lf_train, y_lf_test = train_test_split(df_rating_amenity_train.drop('Tag', axis = 1),\
                                                                                df_rating_amenity_train['Tag'], test_size = 0.3)

In [22]:
from snorkel.labeling import PandasLFApplier

lfs = [lf_rating_star, lf_rating_adj, lf_rating_adj_syn, lf_amenity_lem, lf_amenity_syn, lf_location_prep]

In [23]:
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train_lf_rating_amenity)
L_dev = applier.apply(df=df_test_lf_rating_amenity)

100%|██████████████████████████████████████████████████████████████████████████████| 1390/1390 [00:39<00:00, 35.54it/s]


In [24]:
coverage_stars, coverage_adjectives, coverage_synonyms, coverage_amenity_lem, coverage_amenity_syn, coverage_loc = \
(L_train != ABSTAIN).mean(axis=0)
print(f"Labelling function stars coverage: {coverage_stars * 100:.1f}%")
print(f"Labelling function adjectives coverage: {coverage_adjectives * 100:.1f}%")
print(f"Labelling function synonyms of adjectives coverage: {coverage_synonyms * 100:.1f}%")
print(f"Labelling function lemmas of amenity coverage: {coverage_amenity_lem * 100:.1f}%")
print(f"Labelling function synonyms of amenity coverage: {coverage_amenity_syn * 100:.1f}%")
print(f"Labelling function synonyms of location coverage: {coverage_loc * 100:.1f}%")

Labelling function stars coverage: 1.6%
Labelling function adjectives coverage: 51.6%
Labelling function synonyms of adjectives coverage: 92.1%
Labelling function lemmas of amenity coverage: 15.4%
Labelling function synonyms of amenity coverage: 28.0%
Labelling function synonyms of location coverage: 23.7%


In [25]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
lf_rating_star,0,[0],0.016343,0.016343,0.008017
lf_rating_adj,1,[0],0.515880,0.515572,0.213383
lf_rating_adj_syn,2,[0],0.921061,0.753006,0.445883
lf_amenity_lem,3,[1],0.154178,0.154178,0.134135
lf_amenity_syn,4,[1],0.279988,0.277212,0.257169
lf_location_prep,5,[2],0.237126,0.233734,0.233734


In [26]:
LFAnalysis(L=L_dev, lfs=lfs).lf_summary(y_lf_test)

ValueError: Classification metrics can't handle a mix of unknown and binary targets